# Magnetotellurics

This object can be used to store magnetotelluric (MT) data - a natural-source geophysical method widely used for deep crustal investigations. Electromagnetic data are provided in the frequency-domain as point source measurments of either impedances or apparent resistity/phase.

The following example shows how to generate an MT survey with associated data programmatically.

![mtSurvey](./images/mt_survey.png)

In [1]:
import numpy as np
from geoh5py.workspace import Workspace
from geoh5py.objects import MTReceivers

# Create a new project
workspace = Workspace("my_project.geoh5")

# Define the receiver locations on 2 lines, 60 m apart
x_loc, y_loc = np.meshgrid(np.linspace(-5, 5, 2), np.linspace(0., 20., 9))
vertices = np.c_[x_loc.ravel(), y_loc.ravel(), np.zeros_like(x_loc).ravel()]

# Create the survey from vertices
mt_survey = MTReceivers.create(workspace, vertices=vertices)

## Metadata

Along with the survey object itself, the metadata contains all the necessary information to define the geophysical experiment.

In [2]:
mt_survey.metadata

{'EM Dataset': {'Channels': [],
  'Input type': 'Rx only',
  'Property groups': [],
  'Receivers': UUID('f0393c8d-1659-41c4-b515-8f0f04574095'),
  'Survey type': 'Magnetotellurics',
  'Unit': 'Hertz (Hz)'}}

### Input type

Generic label used in the `geoh5` standard for all EM survey entities. Restricted to `Rx only` in the case of `MTReceivers` (natural sources method).

### Channels

User must define the frequencies at which the data are provided.

In [4]:
mt_survey.channels = [1., 10., 100.]

## Data

The MT survey expects data to be grouped by component and in the same order as the input channels. The class method `add_component_data` can help add data and set the metadata from nested dictionaries. Below is an example using four 

In [5]:
# Arbitrary data generator using sine functions
data_fun = lambda c, f: (c+1.) * np.sin(f * np.pi * (x_loc * y_loc).ravel() / 200.)

# Create a nested dictionary of component: frequency data.
data = {
    component : {
        freq: {"values": data_fun(cc, ff)} for ff, freq in enumerate(mt_survey.channels)
    } for cc, component in enumerate(["Zxx (real)", "Zxx (imaginary)", "Zxy (real)", "Zyy (real)"])
}
    
mt_survey.add_component_data(data)

**Note:** Not all components of the impedence tensor need to be provided. It is expected however that each component contains channels for all frequencies. The input data channels must be supplied in the same order as defined in the metadata under `survey.channels`.